In [ ]:
from op_analytics.datasources.platform_metrics.pg_daily_pull import PostgresDailyPull

meta = PostgresDailyPull.fetch()
meta

In [ ]:
from op_analytics.datasources.platform_metrics.prometheus_daily_pull import PrometheusDailyPull

meta_prom = PrometheusDailyPull.fetch()
meta_prom

In [2]:
from op_analytics.datasources.platform_metrics import execute

result = execute.execute_pull()

2025-05-20 13:19:14 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=97122
2025-05-20 13:19:14 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=97122


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


2025-05-20 13:19:53 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 0.39 seconds filename=request.py lineno=100 process=97122
2025-05-20 13:19:53 [info     ] Prometheus range query executed duration=0.39s filename=client.py lineno=112 process=97122 query=sum(increase(nat_tests_total[1h]))
2025-05-20 13:19:54 [info     ] DRYRUN WRITE_EMPTY: Wrote 489.4Krows 12.5MB to BQ temp_upserts.platform_metrics_platform_metrics_jobs_v1_202505201019-2b08b0b8 filename=write.py lineno=249 process=97122


    MERGE `platform_metrics.platform_metrics_jobs_v1` T
    USING `temp_upserts.platform_metrics_platform_metrics_jobs_v1_202505201019-2b08b0b8` S
    ON T.date = S.date AND T.id = S.id AND T.pipeline_id = S.pipeline_id AND T.workflow_id = S.workflow_id
    WHEN MATCHED THEN
      UPDATE SET T.workflow_name = S.workflow_name, T.number = S.number, T.name = S.name, T.status = S.status, T.stopped_at = S.stopped_at
    WHEN NOT MATCHED THEN
    

In [ ]:
#Backfill tables
#from op_analytics.coreutils.partitioned.location import DataLocation

# # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
#     def mock_location():
#         return DataLocation.GCS

import os

from op_analytics.coreutils.bigquery.write import (
    overwrite_unpartitioned_table,
)
from op_analytics.coreutils.logger import structlog

from op_analytics.datasources.platform_metrics.pg_daily_pull import PostgresDailyPull
from op_analytics.datasources.platform_metrics.prometheus_daily_pull import PrometheusDailyPull
from op_analytics.coreutils.env.aware import OPLabsEnvironment

log = structlog.get_logger()

# BigQuery Dataset and Tables
BQ_DATASET = "platform_metrics"
ANALYTICS_TABLE_PG = "platform_metrics_jobs_v1"
ANALYTICS_TABLE_PROM = "platform_metrics_prometheus_metrics_v1"

current_env = OPLabsEnvironment.PROD

def write_pg_to_bq(data: PostgresDailyPull):
    """Write to BigQuery.

    This operation will be retired soon. Data will move to GCS + Clickhouse.
    """
    jobs_df = data.jobs_df.rename({"dt": "date"})
    # jobs_df = data.jobs_df_truncated.rename({"dt": "date"})
    
    overwrite_unpartitioned_table(
        df=jobs_df,
        dataset=BQ_DATASET,
        table_name=ANALYTICS_TABLE_PG,
    )

    return {"jobs_df": len(jobs_df)}

def write_prom_to_bq(data: PrometheusDailyPull):
    """Write to BigQuery.

    This operation will be retired soon. Data will move to GCS + Clickhouse.
    """
    metrics_df = data.metrics_df.rename({"dt": "date"})
    # metrics_df = data.metrics_df_truncated.rename({"dt": "date"})

    overwrite_unpartitioned_table(
        df=metrics_df,
        dataset=BQ_DATASET,
        table_name=ANALYTICS_TABLE_PG,
    )

    return {"metrics_df": len(metrics_df)}

data_pg = PostgresDailyPull.fetch()
data_prom = PrometheusDailyPull.fetch()

summary = {}
summary["bigquery"] = {"pg": write_pg_to_bq(data_pg), "prom": write_prom_to_bq(data_prom)}
summary

2025-05-20 14:33:26 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=98910
2025-05-20 14:33:26 [info     ] Initialized Google Cloud SQL connector filename=client.py lineno=32 process=98910


/Users/bellaokiddy/Documents/Github/op-analytics/src/op_analytics/coreutils/postgres/client.py:112: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(data, schema=columns)


2025-05-20 14:35:32 [info     ] Fetched from https://prometheus-blocks-prod-us-central1.grafana.net/api/prom/api/v1/query_range: 0.75 seconds filename=request.py lineno=100 process=98910
2025-05-20 14:35:32 [info     ] Prometheus range query executed duration=0.76s filename=client.py lineno=112 process=98910 query=sum(increase(nat_tests_total[1h]))
2025-05-20 14:35:32 [info     ] DRYRUN WRITE_TRUNCATE: Wrote 489.5Krows 12.5MB to BQ platform_metrics.platform_metrics_jobs_v1 filename=write.py lineno=249 process=98910
2025-05-20 14:35:32 [info     ] DRYRUN WRITE_TRUNCATE: Wrote 13rows 1.7KB to BQ platform_metrics.platform_metrics_jobs_v1 filename=write.py lineno=249 process=98910


{'bigquery': {'pg': {'jobs_df': 489487}, 'prom': {'metrics_df': 13}}}

In [ ]:
import os
from unittest.mock import patch

from op_analytics.coreutils.partitioned import dailydata
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datasources.platform_metrics.dataaccess import Github

data = dfs

def mock_location():
    return DataLocation.GCS

os.environ["ALLOW_WRITE"] = "true"

with patch.object(dailydata, "determine_location", mock_location):
    Github.PRS.write(
        dataframe=data.prs,
        sort_by=["repo", "number", "state", "updated_at"],
    )

    Github.ISSUES.write(
        dataframe=data.issues,
        sort_by=["repo", "number", "state", "updated_at"],
    )

    Github.PR_COMMENTS.write(
        dataframe=data.pr_comments,
        sort_by=["repo", "pr_number", "updated_at"],
    )

    Github.PR_REVIEWS.write(
        dataframe=data.pr_reviews,
        sort_by=["repo", "pr_number", "updated_at"],
    )